In [2]:
!pip install efaqa_corpus_zh
!pip install gensim

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Created wheel for efaqa-corpus-zh: filename=efaqa_corpus_zh-0.2-cp37-none-any.whl size=8667 sha256=67dba4de54c845f81fad7a2de656f2ed0d07ca3f8c014c748aa7b5ba76dbea6f
  Stored in directory: /home/aistudio/.cache/pip/wheels/2e/ca/eb/c6e12293e82bd4ea4b1865e835b0ad202cf8d468d8981cb386
Successfully built efaqa-corpus-zh
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 24.1MB 24.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 35.8MB/s eta 0:00:01


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import efaqa_corpus_zh
#dataset from https://github.com/chatopera/efaqa-corpus-zh
data = list(efaqa_corpus_zh.load('dataset/efaqa-corpus-zh.utf8.gz'))

# Explore the data


In [70]:
num_post = len(data)
num_sentence = sum([len(post["chats"]) for post in data])
avg_num_word = sum(len(chat["value"]) for post in data for chat in post["chats"]) / num_sentence

print("num_post", num_post)
print("num_sentence", num_sentence)
print("each", num_sentence / num_post)
print("average", avg_num_word)

num_post 20000
num_sentence 207745
each 10.38725
average 17.68224987364317


In [71]:
print(data[0])
print(data[500].keys())
print(data[0]["chats"][0]['label'])

{'chats': [{'label': {'knowledge': False, 'negative': False, 'question': True}, 'sender': 'audience', 'time': '11:02:45', 'type': 'textMessage', 'value': '这样的议论是针对谁呢？'}, {'label': {'knowledge': False, 'negative': False, 'question': False}, 'sender': 'audience', 'time': '11:08:38', 'type': 'textMessage', 'value': '我也是一个从小被这样训到大的女生哦，总会被指责缺心少肺、没心眼儿、没眼力见儿、看不出来眉眼高低等等。不过在我成长一段时间之后，发现这件事情其实很简单，也没有什么大的问题。如果你愿意的话，可以找我聊聊，倾诉一下你遇到的事情，希望能够帮到你。我是树洞小太阳，欢迎你来找我玩❤'}, {'label': {'knowledge': False, 'negative': False, 'question': False}, 'sender': 'audience', 'time': '11:15:17', 'type': 'textMessage', 'value': '好惨'}, {'label': {'knowledge': False, 'negative': False, 'question': False}, 'sender': 'audience', 'time': '11:15:35', 'type': 'textMessage', 'value': '原生家庭也这么对你吗'}], 'date': '2020-03-02 11:01:08', 'label': {'s1': '1.13', 's2': '2.7', 's3': '3.4'}, 'owner': '匿名', 'title': '女 听过别人最多的议论就是干啥啥不行不长心眼没有脑子'}
dict_keys(['chats', 'date', 'label', 'owner', 'title'])
{'knowledge': False, 'negative': False, 'qu

# Try ML to do classification

In [30]:
#stop wrods for chinese
!git clone https://github.com/goto456/stopwords.git

正克隆到 'stopwords'...
remote: Enumerating objects: 22, done.
remote: Total 22 (delta 0), reused 0 (delta 0), pack-reused 22
展开对象中: 100% (22/22), 完成.
检查连接... 完成。


In [72]:
#get 20000 post test
from tqdm import tqdm
stop_word_path = "stopwords/baidu_stopwords.txt"
# read stop words
stopwords = [line.strip() for line in open(stop_word_path, 'r', encoding='utf-8').readlines()]

stopwords += [" ", "，", "。"]

len(stopwords)
lines = [[post["title"]] + [chat["value"] for chat in post["chats"]] for post in data]

import jieba
from functools import reduce
from tqdm import tqdm

# seperate word
x = []
for cluster in tqdm(lines):
    x_line = []
    for line in cluster:
        tmp = [char for char in jieba.lcut(line) if char not in stopwords]
        x_line.append(tmp)
    x.append(reduce(lambda a, b: a+b, x_line))




  0%|          | 0/20000 [00:00<?, ?it/s]


  0%|          | 26/20000 [00:00<01:17, 259.21it/s]


  0%|          | 63/20000 [00:00<01:11, 279.81it/s]


  0%|          | 87/20000 [00:00<01:16, 261.99it/s]


  1%|          | 112/20000 [00:00<01:17, 258.17it/s]


  1%|          | 148/20000 [00:00<01:10, 282.05it/s]


  1%|          | 174/20000 [00:00<01:12, 272.39it/s]


  1%|          | 199/20000 [00:00<01:17, 255.98it/s]


  1%|          | 224/20000 [00:00<01:20, 246.71it/s]


  1%|▏         | 252/20000 [00:00<01:17, 255.53it/s]


  1%|▏         | 279/20000 [00:01<01:23, 237.55it/s]


  2%|▏         | 303/20000 [00:01<01:31, 215.70it/s]


  2%|▏         | 327/20000 [00:01<01:29, 219.11it/s]


  2%|▏         | 355/20000 [00:01<01:25, 229.02it/s]


  2%|▏         | 380/20000 [00:01<01:24, 231.33it/s]


  2%|▏         | 404/20000 [00:01<01:32, 211.03it/s]


  2%|▏         | 431/20000 [00:01<01:28, 222.12it/s]


  2%|▏         | 455/20000 [00:01<01:26, 227.17it/s]


  2%|▏         | 484/

In [7]:
#get label for 3 category of 20000 post
from tqdm import tqdm

# predict label
y_s1_raw = []
y_s2_raw = []
y_s3_raw = []

for post in tqdm(data):
    cluster = {item[0]: item[1] for item in post["label"].items()} 
    y_s1_raw.append(cluster["s1"])
    y_s2_raw.append(cluster["s2"])
    y_s3_raw.append(cluster["s3"])

#mapping
y_map = {}
for label in y_s1_raw + y_s2_raw + y_s3_raw:
    if label not in y_map:
        y_map[label] = len(y_map)

print(y_map)

y_s1 = [y_map[label] for label in y_s1_raw]
y_s2 = [y_map[label] for label in y_s2_raw]
y_s3 = [y_map[label] for label in y_s3_raw]
print(len(y_s1),len(y_s2),len(y_s3))
print(y_s1[0:5])

from collections import Counter

#imbalance
print( Counter(y_s1)   )
print( Counter(y_s2)   )
print( Counter(y_s3)   )

100%|██████████| 20000/20000 [00:00<00:00, 338145.33it/s]


{'1.13': 0, '1.16': 1, '1.6': 2, '1.9': 3, '1.14': 4, '1.7': 5, '1.12': 6, '1.3': 7, '1.15': 8, '1.8': 9, '1.2': 10, '1.1': 11, '1.10': 12, '1.11': 13, '1.4': 14, '1.5': 15, '1.18': 16, '1.17': 17, '1.19': 18, '2.7': 19, '2.1': 20, '2.2': 21, '2.8': 22, '2.3': 23, '2.4': 24, '2.5': 25, '2.6': 26, '3.4': 27, '3.2': 28, '3.3': 29, '3.6': 30, '3.5': 31}
20000 20000 20000
[0, 1, 1, 1, 1]
Counter({1: 7156, 3: 4968, 7: 1979, 5: 879, 9: 817, 13: 728, 11: 659, 0: 511, 6: 444, 2: 406, 12: 403, 10: 294, 8: 202, 4: 195, 14: 104, 18: 87, 16: 71, 17: 53, 15: 44})
Counter({19: 17799, 20: 907, 21: 716, 24: 236, 22: 146, 23: 94, 25: 69, 26: 33})
Counter({27: 19612, 28: 247, 29: 88, 30: 49, 31: 4})


In [67]:
from sklearn.model_selection import train_test_split

x1_train, x1_test, y1_train, y1_test \
        = train_test_split(x, y_s1, test_size=0.3)
x2_train, x2_test, y2_train, y2_test \
        = train_test_split(x, y_s2, test_size=0.3)
x3_train, x3_test, y3_train, y3_test \
        = train_test_split(x, y_s3, test_size=0.3)


In [54]:
import gensim
# print(token_embedding.search(x_train[0]))
word2vec_model = gensim.models.Word2Vec(x1_train)

In [55]:
def get_doc_vec(x, word2vec_model):
    doc_vec_s1 = []
    zero_count = 0
    for doc in tqdm(x):
        tmp = [word2vec_model.wv[word] for word in doc if word in word2vec_model.wv]
        if len(tmp) == 0:
            avg = np.zeros(len(doc_vec_s1[0]))
            zero_count += 1
        else:
            avg = [item/len(tmp) for item in reduce(lambda lt1, lt2: [lt1[index]+lt2[index] for index in range(len(lt1))], tmp)]
        doc_vec_s1.append(avg)
    print("zero vector", zero_count/len(doc_vec_s1))
    return doc_vec_s1

X1_train = get_doc_vec(x1_train, word2vec_model)
X1_test  = get_doc_vec(x1_test, word2vec_model)




  0%|          | 0/14000 [00:00<?, ?it/s]


  0%|          | 38/14000 [00:00<00:36, 379.01it/s]


  1%|          | 85/14000 [00:00<00:34, 400.07it/s]


  1%|          | 143/14000 [00:00<00:31, 440.23it/s]


  1%|▏         | 186/14000 [00:00<00:31, 436.16it/s]


  2%|▏         | 242/14000 [00:00<00:29, 466.57it/s]


  2%|▏         | 284/14000 [00:00<00:34, 399.10it/s]


  2%|▏         | 342/14000 [00:00<00:31, 439.68it/s]


  3%|▎         | 394/14000 [00:00<00:29, 458.01it/s]


  3%|▎         | 447/14000 [00:00<00:28, 476.78it/s]


  4%|▎         | 502/14000 [00:01<00:27, 495.08it/s]


  4%|▍         | 562/14000 [00:01<00:25, 522.45it/s]


  4%|▍         | 617/14000 [00:01<00:25, 530.09it/s]


  5%|▍         | 671/14000 [00:01<00:25, 524.26it/s]


  5%|▌         | 727/14000 [00:01<00:24, 532.29it/s]


  6%|▌         | 781/14000 [00:01<00:24, 532.15it/s]


  6%|▌         | 835/14000 [00:01<00:25, 525.71it/s]


  6%|▋         | 888/14000 [00:01<00:25, 507.21it/s]


  7%|▋         | 939

zero vector 0.0





  0%|          | 0/6000 [00:00<?, ?it/s]


  1%|          | 70/6000 [00:00<00:08, 694.79it/s]


  2%|▏         | 135/6000 [00:00<00:08, 679.69it/s]


  3%|▎         | 189/6000 [00:00<00:09, 630.58it/s]


  4%|▍         | 250/6000 [00:00<00:09, 623.21it/s]


  5%|▌         | 304/6000 [00:00<00:09, 593.27it/s]


  6%|▌         | 358/6000 [00:00<00:09, 574.84it/s]


  7%|▋         | 423/6000 [00:00<00:09, 592.62it/s]


  8%|▊         | 478/6000 [00:00<00:09, 571.93it/s]


  9%|▉         | 533/6000 [00:00<00:09, 562.24it/s]


 10%|▉         | 592/6000 [00:01<00:09, 569.28it/s]


 11%|█         | 648/6000 [00:01<00:09, 553.33it/s]


 12%|█▏        | 703/6000 [00:01<00:09, 549.94it/s]


 13%|█▎        | 759/6000 [00:01<00:09, 551.20it/s]


 14%|█▎        | 814/6000 [00:01<00:09, 543.47it/s]


 14%|█▍        | 869/6000 [00:01<00:09, 530.81it/s]


 15%|█▌        | 926/6000 [00:01<00:09, 536.56it/s]


 16%|█▋        | 980/6000 [00:01<00:09, 529.04it/s]


 17%|█▋        | 1033/6000 [00:01<00:

zero vector 0.00016666666666666666


In [60]:
print(X1_test[0])

[0.16836156564600327, 0.11222276968114517, -0.15600928138284123, 0.235136116252226, -0.04517208478030037, -0.431303949917064, 0.10600343872519101, 0.40251027836519127, -0.549279717838063, -0.15212563907398896, -0.07575958616593305, -0.1997673511505127, -0.2436065393335679, 0.12940189417670755, -0.11400430342730354, -0.030603380764231962, -0.081016645712011, 0.014301672577857971, -0.1463780683629653, -0.46513927684110756, 0.14229583740234375, 0.26988568025476795, 0.33950003455666933, -0.16803937799790325, -0.019632069503559786, -0.13160213302163518, 0.1048821491353652, 0.0620114242329317, -0.15555439275853775, 0.09759168063893038, 0.3518732014824362, -0.0008335030254195718, 0.2825167880338781, 0.002122400438084322, -0.22092147434459014, 0.4370293617248535, 0.29839176290175495, -0.1534134640413172, -0.17386193836436553, -0.4525494855992934, -0.08144584824057187, 0.18196762309354894, -0.017369648989509132, 0.21601191688986385, 0.19030141830444336, 0.15270019980037913, 0.020697572652031395

In [40]:
#LR
from sklearn.linear_model import LogisticRegression
LogisticRegression = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0,
    fit_intercept=True, intercept_scaling=1, class_weight=None,
    random_state=None, solver='liblinear', max_iter=100, multi_class='ovr',
    verbose=0, warm_start=False, n_jobs=1)

#bayes
from sklearn import naive_bayes
GaussianNB=naive_bayes.GaussianNB() # 高斯贝叶斯

MultinomialNB=naive_bayes.MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)

BernoulliNB= naive_bayes.BernoulliNB(alpha=1.0, binarize=0.0, fit_prior=True, class_prior=None)

#DT
from sklearn import tree
DecisionTree = tree.DecisionTreeClassifier(criterion='gini', max_depth=None,
    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
    max_features=None, random_state=None, max_leaf_nodes=None,
    min_impurity_decrease=0.0, min_impurity_split=None,
     class_weight=None)
                  
#svc
from sklearn.svm import SVC
SVC = SVC(C=1.0, kernel='rbf', gamma='auto')
                  
#kNN
from sklearn import neighbors                 
KNN = neighbors.KNeighborsClassifier(n_neighbors=5, n_jobs=1) # 分类
                  
#MLP               
from sklearn.neural_network import MLPClassifier
MLPClassifier = MLPClassifier(activation='relu', solver='adam', alpha=0.0001,max_iter=200)
                  
#rf
from sklearn.ensemble import RandomForestClassifier
RandomForest = RandomForestClassifier()

In [62]:
#select target model
cls_model=[]
cls_model.append(('Logistic Regression:', LogisticRegression))
cls_model.append(('GaussianNB :', GaussianNB))
# cls_model.append(('MultinomialNB      :', MultinomialNB))
cls_model.append(('BernoulliNB     :', BernoulliNB))
cls_model.append(('DecisionTree :', DecisionTree))
cls_model.append(('SVC :', SVC))   
cls_model.append(('KNN :', KNN)) 
cls_model.append(('MLPClassifier :', MLPClassifier))
cls_model.append(('RandomForest :', RandomForest)) 
print(len(cls_model))

8


In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score, classification_report

for name, model in cls_model:
    model.fit(X1_train, y1_train)
    pred = model.predict(X1_test).astype(int)
    print(name,accuracy_score(y1_test, pred))
    print(classification_report(y1_test, pred))

Logistic Regression: 0.5856666666666667
              precision    recall  f1-score   support

           0       0.44      0.18      0.25       157
           1       0.59      0.81      0.68      2191
           2       0.65      0.30      0.41       121
           3       0.61      0.82      0.70      1460
           4       0.00      0.00      0.00        55
           5       0.47      0.17      0.26       269
           6       0.00      0.00      0.00       134
           7       0.53      0.43      0.47       560
           8       0.00      0.00      0.00        58
           9       0.49      0.26      0.34       254
          10       0.44      0.21      0.28        77
          11       0.66      0.46      0.54       206
          12       0.39      0.06      0.10       127
          13       0.61      0.11      0.18       208
          14       0.00      0.00      0.00        36
          15       0.00      0.00      0.00        14
          16       0.00      0.00      0.

In [17]:
#try DL
idx=1
lines = [[post["title"]] + [chat["value"] for chat in post["chats"]] for post in data]

x=[]
for i in lines:
    tmp=''
    for j in i:
        tmp+=j
    x.append(tmp)

print(x[0].replace(' ',''))
print(y_s1[0])

with open("trainmh.txt","w") as f:
    for i in range(0,20000):
        ans=str(idx)+'\t'+x[i].replace(' ','').replace('\t','').replace('\n','').replace('\r','')+'\t'+str(y_s1[i])
        idx+=1
        f.write(ans+'\n')
    print('done')
f.close()

女听过别人最多的议论就是干啥啥不行不长心眼没有脑子这样的议论是针对谁呢？我也是一个从小被这样训到大的女生哦，总会被指责缺心少肺、没心眼儿、没眼力见儿、看不出来眉眼高低等等。不过在我成长一段时间之后，发现这件事情其实很简单，也没有什么大的问题。如果你愿意的话，可以找我聊聊，倾诉一下你遇到的事情，希望能够帮到你。我是树洞小太阳，欢迎你来找我玩❤好惨原生家庭也这么对你吗
0
done


In [19]:
idx=1
with open("trainmh.txt", 'r', encoding='utf-8') as fp:
    for line in fp.readlines():
        try:
            order, words, labels = line.strip('\n').split('\t')
        except:
            print(line)
        idx+=1

In [20]:
x[697]

'我外表上被看作是一个很乖很单纯的男孩，但其实我内心邪恶，只有我自己知道，感觉我就是披着羊皮的狼，?\r\n此话怎讲怎么了发生什么了内心怎么邪恶的啊'

In [24]:
import paddlehub as hub

module = hub.Module(name="ernie_gen")

result = module.finetune(
    train_path='trainmh.txt',
    save_dir="ernie_gen_ch_med",
    max_steps=4000,
    max_encode_len=256,
    max_decode_len=256,
    noise_prob=0.1,
    batch_size=16,
    log_interval=50,
    save_interval=2000
)

[2021-10-24 21:29:01,990] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2021-10-24 21:29:02,107] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2021-10-24 21:29:02,110] [   DEBUG] - init ErnieModel with config: {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'relu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'max_position_embeddings': 513, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 18000, 'pad_token_id': 0}
[2021-10-24 21:29:02,205] [    INFO] - loading pretrained model from /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2021-10-24 21:29:02,898] [    INFO] - param:mlm_bias not set in pretrained model, skip
[2021-10-24 21:29:02,900] [    INFO] - param:mlm.weight not set in pretrained model, skip
[2021-10-24 21:29:02,904] [    INFO] - param:mlm.bias not set in pretrained model, skip
[2021-10-24 21:

In [25]:
import paddlehub as hub

module = hub.Module(name="ernie_gen")
module.export(params_path='ernie_gen_ch_med/step_4000_ppl_1.24646.params', module_name="ernie_gen_efaqa", author="vemodalen")

[2021-10-24 21:56:41,278] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2021-10-24 21:56:41,296] [    INFO] - Begin export the model save in ernie_gen_ch_med/step_4000_ppl_1.24646.params ...
[2021-10-24 21:56:41,787] [    INFO] - The module has exported to /home/aistudio/ernie_gen_efaqa


In [27]:
!hub install ernie_gen_efaqa

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
[2021-10-24 21:57:17,512] [    INFO] - Successfully installed ernie_gen_efaqa-1.0.0


In [13]:
import paddlehub as hub
import os

os.environ["CUDA_VISIBLE_DEVICES"] ="0"

module = hub.Module(name="ernie_gen_efaqa")

test_texts = ["哇啊啊啊啊啊啊我学的好累了学不动了感觉要抑郁了"]
results = module.generate(texts=test_texts, use_gpu=True, beam_width=5)
for result in results:
    print(result)

[2021-10-24 22:15:29,803] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2021-10-24 22:15:29,805] [   DEBUG] - init ErnieModel with config: {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'relu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'max_position_embeddings': 513, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 18000, 'pad_token_id': 0}
[2021-10-24 22:15:29,895] [    INFO] - loading pretrained model from /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2021-10-24 22:15:30,394] [    INFO] - param:mlm_bias not set in pretrained model, skip
[2021-10-24 22:15:30,397] [    INFO] - param:mlm.weight not set in pretrained model, skip
[2021-10-24 22:15:30,399] [    INFO] - param:mlm.bias not set in pretrained model, skip
[2021-10-24 22:15:30,401] [    INFO] - param:mlm_ln.weight not set in pretrained model, skip
[2021-10-24 22:15:30,403] [  

['11', '1', '5', '6', '4']


# FINETUNE WITH SKEP ERUINE

In [45]:
from sklearn.model_selection import train_test_split
import efaqa_corpus_zh
data = list(efaqa_corpus_zh.load('dataset/efaqa-corpus-zh.utf8.gz'))

lines = [[post["title"]] + [chat["value"] for chat in post["chats"]] for post in data]

x=[]
for i in lines:
    tmp=''
    for j in i:
        tmp+=j
    x.append(tmp)

#get label for 3 category of 20000 post
from tqdm import tqdm

# predict label
y_s1_raw = []
y_s2_raw = []
y_s3_raw = []

for post in tqdm(data):
    cluster = {item[0]: item[1] for item in post["label"].items()} 
    y_s1_raw.append(cluster["s1"])
    y_s2_raw.append(cluster["s2"])
    y_s3_raw.append(cluster["s3"])

#mapping
y_map = {}
for label in y_s1_raw + y_s2_raw + y_s3_raw:
    if label not in y_map:
        y_map[label] = len(y_map)

print(y_map)

y_s1 = [y_map[label] for label in y_s1_raw]
y_s2 = [y_map[label] for label in y_s2_raw]
y_s3 = [y_map[label] for label in y_s3_raw]
print(len(y_s1),len(y_s2),len(y_s3))
print(y_s1[0:5])

from collections import Counter

#imbalance
print( Counter(y_s1)   )
print( Counter(y_s2)   )
print( Counter(y_s3)   )

x1_train, x1_test, y1_train, y1_test \
        = train_test_split(x, y_s1, test_size=0.3)
x2_train, x2_test, y2_train, y2_test \
        = train_test_split(x, y_s2, test_size=0.3)
x3_train, x3_test, y3_train, y3_test \
        = train_test_split(x, y_s3, test_size=0.3)

100%|██████████| 20000/20000 [00:00<00:00, 258723.19it/s]


{'1.13': 0, '1.16': 1, '1.6': 2, '1.9': 3, '1.14': 4, '1.7': 5, '1.12': 6, '1.3': 7, '1.15': 8, '1.8': 9, '1.2': 10, '1.1': 11, '1.10': 12, '1.11': 13, '1.4': 14, '1.5': 15, '1.18': 16, '1.17': 17, '1.19': 18, '2.7': 19, '2.1': 20, '2.2': 21, '2.8': 22, '2.3': 23, '2.4': 24, '2.5': 25, '2.6': 26, '3.4': 27, '3.2': 28, '3.3': 29, '3.6': 30, '3.5': 31}
20000 20000 20000
[0, 1, 1, 1, 1]
Counter({1: 7156, 3: 4968, 7: 1979, 5: 879, 9: 817, 13: 728, 11: 659, 0: 511, 6: 444, 2: 406, 12: 403, 10: 294, 8: 202, 4: 195, 14: 104, 18: 87, 16: 71, 17: 53, 15: 44})
Counter({19: 17799, 20: 907, 21: 716, 24: 236, 22: 146, 23: 94, 25: 69, 26: 33})
Counter({27: 19612, 28: 247, 29: 88, 30: 49, 31: 4})


In [46]:
train_ds=[]
dev_ds=[]

for i in range(0,len(y1_train)):
    tmp={'text':x1_train[i],'label':int(y1_train[i]),'qid':''}
    train_ds.append(tmp)

for i in range(0,len(y1_test)):
    tmp={'text':x1_test[i],'label':int(y1_test[i]),'qid':''}
    dev_ds.append(tmp)

print(train_ds[0])
print(dev_ds[1])


{'text': '女 这样是否有抑郁症？它这个我看不懂，麻烦了。我真的感觉生活没有希望。刚刚看完您所有的描述，scl～90这个测试有两个因子分，一个是阴性因子，另一个是阳性因子，只要关注下有症状的阳性因子分就好。\n\n那么所有＞＝2分的都属于阳性因子分。（上限分值是5，也就是最高5分是说明症状最严重。）\n\n不过，通常测试无法做诊断，仅供参考，更多的需要根据您生活中近期的状态相结合去做评估。s\tc\tl-90是从10个方面了解自己心理健康程度的1种自评量表。\n1、反应的10个方面分别为:\n（1）躯体化（2）强迫症状\n（3）人际关系敏感\n（4）抑郁（5）焦虑（6）敌对\n（7）恐怖（8）偏执（9）精神病性\n（10）其他，如睡眠饮食状况等\n2、其中包括10个因子，每1个因子反映出个体某1方面的症状情况，通过因子分可了解症状分布特点。\n3、如何评估:\n（1）\t总分:\t超过160分\n（2）阳性项目数:\t超过43项\n（3）单个因子分:超过2分\n出现以上任意情况，就可考虑筛选阳性，需要进1步检查。测试条目2分以上显示有些困扰喔，有一定的抑郁情绪呢这个测试表面你有较重的抑郁情绪，不能说是否有抑郁症测试有很好的参考价值，但不做确诊依据。你生活中发生了什么？我家庭不是很完整和妈妈生活但是过的很累一周有六天都在和妈妈吵架\n妈妈经常的一些举措让我感到害怕和妈妈没有好好说过话她也从来没有理解过我我基本有两天或者三天在深夜里或者白天都会哭我发现我现在真的改变了好多有时候没有原因但就是会难过对生活也没希望总是很烦躁。有时候睡不着觉我也时常有那些想法', 'label': 7, 'qid': ''}
{'text': '我请问有什么可以帮你？欢迎你点我头像加关注有什么事情可以随时私聊我你可以点击关注我这里很安全我从七岁就得了癫痫病，一直吃药控制，因而导致阴茎短小，觉得自己低人一等，在性方面有欲望，但是在性生活时就勃而不坚，因此与女朋友分手，现在我总觉得自己活下去就是别人的笑柄可以点击咨询师头像，加关注，私聊，具体帮你分析确诊，咨询治疗。最好通过平台电话联系我咨询。', 'label': 1, 'qid': ''}


In [47]:
from paddlenlp.datasets import MapDataset
train_ds=MapDataset(train_ds)
dev_ds=MapDataset(dev_ds)

print(train_ds[5])
print(dev_ds[10])

{'text': '有没有美女老师，想咨询下！心理变态。喜欢黑色丝袜的女人你多大了这种情况多长时间了24岁有2年了吧哦 这个问题让你有精神上的痛苦吗没有人性都是追求快乐，逃离痛苦，变态心理的起因是因为任自己欲望随心所遇，最后收不住脚。', 'label': 6, 'qid': ''}
{'text': '女 小时候被暴露狂吓到过导致长大后对于人体X器官极为敏感一跟人交流就会手心冒汗焦虑满满的嫌弃感注意力老是觉得人体X性器官脏请问这种心理怎样克服至于吗一言难尽可是你以后得交男朋友，那避免不了有性生活的不要觉得脏不过确实也脏这个无法自我克服，需要通过咨询来处理当时给您留下的心结。心结处理好了，以后就不会影响您了。当时的场景带来的心灵创伤，需要重新认识。你这种事情有解决办法，我点我私聊吧过去引发的恐怖情绪还在今天困扰您。私聊', 'label': 9, 'qid': ''}


In [4]:
from paddlenlp.transformers import SkepForSequenceClassification, SkepTokenizer

# 指定模型名称，一键加载模型
model = SkepForSequenceClassification.from_pretrained(pretrained_model_name_or_path="skep_ernie_1.0_large_ch", num_classes=19)
# 同样地，通过指定模型名称一键加载对应的Tokenizer，用于处理文本数据，如切分token，转token_id等。
tokenizer = SkepTokenizer.from_pretrained(pretrained_model_name_or_path="skep_ernie_1.0_large_ch")

[2021-10-24 23:12:16,270] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/skep_ernie_1.0_large_ch/skep_ernie_1.0_large_ch.pdparams
[2021-10-24 23:12:24,765] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/skep_ernie_1.0_large_ch/skep_ernie_1.0_large_ch.vocab.txt


In [10]:
import os
from functools import partial

import numpy as np
import paddle
import paddle.nn.functional as F
from paddlenlp.data import Stack, Tuple, Pad

def convert_example(example,
                    tokenizer,
                    max_seq_length=512,
                    is_test=False):
    encoded_inputs = tokenizer(
        text=example["text"], max_seq_len=max_seq_length)


    input_ids = encoded_inputs["input_ids"]
    # token_type_ids
    token_type_ids = encoded_inputs["token_type_ids"]

    if not is_test:
        # label
        label = np.array([example["label"]], dtype="int64")
        return input_ids, token_type_ids, label
    else:
        # qid = np.array([example["qid"]], dtype="int64")
        return input_ids, token_type_ids

def create_dataloader(dataset,
                      trans_fn=None,
                      mode='train',
                      batch_size=1,
                      batchify_fn=None):
    if trans_fn:
        dataset = dataset.map(trans_fn)

    shuffle = True if mode == 'train' else False
    if mode == "train":
        sampler = paddle.io.DistributedBatchSampler(
            dataset=dataset, batch_size=batch_size, shuffle=shuffle)
    else:
        sampler = paddle.io.BatchSampler(
            dataset=dataset, batch_size=batch_size, shuffle=shuffle)
    dataloader = paddle.io.DataLoader(
        dataset, batch_sampler=sampler, collate_fn=batchify_fn)
    return dataloader

In [11]:
batch_size = 16
# max_seq_length
max_seq_length = 256

trans_func = partial(
    convert_example,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length)

batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # token_type_ids
    Stack()  # labels
): [data for data in fn(samples)]
train_data_loader = create_dataloader(
    train_ds,
    mode='train',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func)
dev_data_loader = create_dataloader(
    dev_ds,
    mode='dev',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func)

In [ ]:
import time

@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader):
    """
    Given a dataset, it evals model and computes the metric.

    Args:
        model(obj:`paddle.nn.Layer`): A model to classify texts.
        criterion(obj:`paddle.nn.Layer`): It can compute the loss.
        metric(obj:`paddle.metric.Metric`): The evaluation metric.
        data_loader(obj:`paddle.io.DataLoader`): The dataset loader which generates batches.
    """
    model.eval()
    metric.reset()
    losses = []
    for batch in data_loader:
        input_ids, token_type_ids, labels = batch
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        losses.append(loss.numpy())
        correct = metric.compute(logits, labels)
        metric.update(correct)
        accu = metric.accumulate()
    print("eval loss: %.5f, accu: %.5f" % (np.mean(losses), accu))
    model.train()
    metric.reset()

# setup
epochs = 20
# save
ckpt_dir = "skep_ckpt"
num_training_steps = len(train_data_loader) * epochs

# optimizer
optimizer = paddle.optimizer.AdamW(
    learning_rate=2e-5,
    parameters=model.parameters())
# loss
criterion = paddle.nn.loss.CrossEntropyLoss()
# accuracy
metric = paddle.metric.Accuracy()

In [ ]:
# train
global_step = 0
tic_train = time.time()
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch
        logits = model(input_ids, token_type_ids)
        # loss
        loss = criterion(logits, labels)
        probs = F.softmax(logits, axis=1)
        # acc
        correct = metric.compute(probs, labels)
        metric.update(correct)
        acc = metric.accumulate()

        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, acc,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # bp
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()

        if global_step % 100 == 0:
            save_dir = os.path.join(ckpt_dir, "model_%d" % global_step)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            # evaluate model
            evaluate(model, criterion, metric, dev_data_loader)
            # save
            model.save_pretrained(save_dir)
            # save
            tokenizer.save_pretrained(save_dir)

global step 10, epoch: 1, batch: 10, loss: 2.29342, accu: 0.26250, speed: 1.33 step/s
global step 20, epoch: 1, batch: 20, loss: 1.83934, accu: 0.36250, speed: 1.32 step/s
global step 30, epoch: 1, batch: 30, loss: 1.79091, accu: 0.36250, speed: 1.31 step/s
global step 40, epoch: 1, batch: 40, loss: 1.95799, accu: 0.38281, speed: 1.30 step/s
global step 50, epoch: 1, batch: 50, loss: 2.03728, accu: 0.38250, speed: 1.29 step/s
global step 60, epoch: 1, batch: 60, loss: 1.98817, accu: 0.39687, speed: 1.30 step/s
global step 70, epoch: 1, batch: 70, loss: 1.23931, accu: 0.41786, speed: 1.30 step/s
global step 80, epoch: 1, batch: 80, loss: 1.09943, accu: 0.43828, speed: 1.29 step/s
global step 90, epoch: 1, batch: 90, loss: 1.81883, accu: 0.45139, speed: 1.29 step/s
global step 100, epoch: 1, batch: 100, loss: 1.14124, accu: 0.46500, speed: 1.29 step/s
eval loss: 1.35201, accu: 0.57983
global step 110, epoch: 1, batch: 110, loss: 1.59147, accu: 0.56250, speed: 0.08 step/s
global step 120,

# ERNIE

In [ ]:
#pip install paddle-ernie

In [48]:
#TRY ERNIE
import paddlenlp as ppnlp
MODEL_NAME = "ernie-1.0"

tokenizer = ppnlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME)
ernie_model = ppnlp.transformers.ErnieModel.from_pretrained(MODEL_NAME)

from functools import partial
from paddlenlp.data import Stack, Tuple, Pad

# 模型运行批处理大小
batch_size = 32
max_seq_length = 256

trans_func = partial(
    convert_example,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length)
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment
    Stack(dtype="int64")  # label
): [data for data in fn(samples)]
train_data_loader = create_dataloader(
    train_ds,
    mode='train',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func)
dev_data_loader = create_dataloader(
    dev_ds,
    mode='dev',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func)

model = ppnlp.transformers.ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=19)

[2021-10-25 12:38:02,271] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2021-10-25 12:38:02,295] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2021-10-25 12:38:03,283] [    INFO] - Weights from pretrained model not used in ErnieModel: ['cls.predictions.layer_norm.weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight', 'cls.predictions.layer_norm.bias']
[2021-10-25 12:38:03,588] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams


In [49]:
from paddlenlp.transformers import LinearDecayWithWarmup

# lr
learning_rate = 5e-5 
# epoech
epochs = 20 
# warm_up
warmup_proportion = 0.1
# weight_decay
weight_decay = 0.01

num_training_steps = len(train_data_loader) * epochs
lr_scheduler = LinearDecayWithWarmup(learning_rate, num_training_steps, warmup_proportion)
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ])

criterion = paddle.nn.loss.CrossEntropyLoss()
metric = paddle.metric.Accuracy()

In [51]:
import paddle.nn.functional as F

ckpt_dir = "ernie_ckpt"
global_step = 0
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, segment_ids, labels = batch
        logits = model(input_ids, segment_ids)
        loss = criterion(logits, labels)
        probs = F.softmax(logits, axis=1)
        correct = metric.compute(probs, labels)
        metric.update(correct)
        acc = metric.accumulate()

        global_step += 1
        if global_step % 10 == 0 :
            print("global step %d, epoch: %d, batch: %d, loss: %.5f, acc: %.5f" % (global_step, epoch, step, loss, acc))
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()
    if epoch % 1 == 0:
        save_dir = os.path.join(ckpt_dir, "model_%d" % global_step)
        if not os.path.exists(save_dir):
             os.makedirs(save_dir)
        # evaluate model
        evaluate(model, criterion, metric, dev_data_loader)
        # save
        model.save_pretrained(save_dir)
        # save
        tokenizer.save_pretrained(save_dir)

global step 10, epoch: 1, batch: 10, loss: 2.95865, acc: 0.09007
global step 20, epoch: 1, batch: 20, loss: 2.91498, acc: 0.09730
global step 30, epoch: 1, batch: 30, loss: 2.62086, acc: 0.11516
global step 40, epoch: 1, batch: 40, loss: 2.81059, acc: 0.13477
global step 50, epoch: 1, batch: 50, loss: 2.27408, acc: 0.15583
global step 60, epoch: 1, batch: 60, loss: 2.55199, acc: 0.17671
global step 70, epoch: 1, batch: 70, loss: 1.92069, acc: 0.20013
global step 80, epoch: 1, batch: 80, loss: 2.01599, acc: 0.21935
global step 90, epoch: 1, batch: 90, loss: 2.30413, acc: 0.23163
global step 100, epoch: 1, batch: 100, loss: 2.09712, acc: 0.24597
global step 110, epoch: 1, batch: 110, loss: 2.09347, acc: 0.26329
global step 120, epoch: 1, batch: 120, loss: 2.12261, acc: 0.27951
global step 130, epoch: 1, batch: 130, loss: 2.20095, acc: 0.29383
global step 140, epoch: 1, batch: 140, loss: 2.22322, acc: 0.30316
global step 150, epoch: 1, batch: 150, loss: 1.85222, acc: 0.31699
global step 1

In [52]:
def predict(model, data, tokenizer, label_map, batch_size=1):
    """
    Predicts the data labels.

    Args:
        model (obj:`paddle.nn.Layer`): A model to classify texts.
        data (obj:`List(Example)`): The processed data whose each element is a Example (numedtuple) object.
            A Example object contains `text`(word_ids) and `se_len`(sequence length).
        tokenizer(obj:`PretrainedTokenizer`): This tokenizer inherits from :class:`~paddlenlp.transformers.PretrainedTokenizer` 
            which contains most of the methods. Users should refer to the superclass for more information regarding methods.
        label_map(obj:`dict`): The label id (key) to label str (value) map.
        batch_size(obj:`int`, defaults to 1): The number of batch.

    Returns:
        results(obj:`dict`): All the predictions labels.
    """
    examples = []
    for text in data:
        input_ids, segment_ids = convert_example(
            text,
            tokenizer,
            max_seq_length=128,
            is_test=True)
        examples.append((input_ids, segment_ids))

    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input id
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # segment id
    ): fn(samples)

    # Seperates data into some batches.
    batches = []
    one_batch = []
    for example in examples:
        one_batch.append(example)
        if len(one_batch) == batch_size:
            batches.append(one_batch)
            one_batch = []
    if one_batch:
        # The last batch whose size is less than the config batch_size setting.
        batches.append(one_batch)

    results = []
    model.eval()
    for batch in batches:
        input_ids, segment_ids = batchify_fn(batch)
        input_ids = paddle.to_tensor(input_ids)
        segment_ids = paddle.to_tensor(segment_ids)
        logits = model(input_ids, segment_ids)
        probs = F.softmax(logits, axis=1)
        idx = paddle.argmax(probs, axis=1).numpy()
        idx = idx.tolist()
        labels = [label_map[i] for i in idx]
        results.extend(labels)
    return results


In [54]:
data = [
    {"text":'我感觉我好垃圾','qid':''},
    {"text":'我被那个坏女人伤透了心','qid':''},
    {"text":'太难了学不下去了','qid':''},
     {"text":'哎我晚上一直睡不着怎么办呀','qid':''},
]
label_map = {'低自尊': 0, '其他': 1, '失眠': 2, '情感关系问题': 3, '青春期问题': 4, '压力': 5, '自我探索': 6, '家庭问题和矛盾': 7, '强迫症': 8, '人际关系': 9, '事业和工作烦恼': 10,
 '学业烦恼、对未来规划的迷茫': 11, '离婚': 12, '分手': 13, '物质滥用': 14, '悲恸': 15, '性问题': 16, 'LGBT': 17, '亲子关系': 18, '尚未到达S2': 19, '忧郁症': 20, 
 '焦虑症': 21, '其他疾病': 22, '躁郁症': 23, '创伤后应激反应': 24, '恐慌症': 25, '厌食症和暴食症': 26, '进行的人身伤害': 27, '策划及逆行的自杀行为': 28, '自残': 29, '无伤害身体倾向': 30, '计划的人身伤害': 31}
label_map = {value:key for key,value in label_map.items()}
print(label_map)

results = predict(
    model, data, tokenizer, label_map, batch_size=batch_size)
for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text, results[idx]))

{0: '低自尊', 1: '其他', 2: '失眠', 3: '情感关系问题', 4: '青春期问题', 5: '压力', 6: '自我探索', 7: '家庭问题和矛盾', 8: '强迫症', 9: '人际关系', 10: '事业和工作烦恼', 11: '学业烦恼、对未来规划的迷茫', 12: '离婚', 13: '分手', 14: '物质滥用', 15: '悲恸', 16: '性问题', 17: 'LGBT', 18: '亲子关系', 19: '尚未到达S2', 20: '忧郁症', 21: '焦虑症', 22: '其他疾病', 23: '躁郁症', 24: '创伤后应激反应', 25: '恐慌症', 26: '厌食症和暴食症', 27: '进行的人身伤害', 28: '策划及逆行的自杀行为', 29: '自残', 30: '无伤害身体倾向', 31: '计划的人身伤害'}
Data: {'text': '我感觉我好垃圾', 'qid': ''} 	 Lable: 其他
Data: {'text': '我被那个坏女人伤透了心', 'qid': ''} 	 Lable: 情感关系问题
Data: {'text': '太难了学不下去了', 'qid': ''} 	 Lable: 学业烦恼、对未来规划的迷茫
Data: {'text': '哎我晚上一直睡不着怎么办呀', 'qid': ''} 	 Lable: 失眠
